In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor, RandomForestRegressor
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

> Считаем данные

In [ ]:
train_data = pd.read_csv("../input/mai-ml-hw-models/Train.csv")
test_data = pd.read_csv("../input/mai-ml-hw-models/Test.csv")
sample=pd.read_csv("/kaggle/input/mai-ml-hw-models/SampleSubmission.csv")

# https://github.com/Dyakonov/python_hacks/blob/master/dj_cat_coding.ipynb
print(train_data.columns)
train_data.head()

# Подготовка данных:
# 1. Избавиться от даты
# 2. Заполнить пропущенные значения
# 3. Нагенерить новых фич
# 4. Избавиться от выбросов

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(x = train_data['street_id'], y = train_data['price'])
plt.ylabel('price', fontsize=12)
plt.xlabel('street_id', fontsize=12)
plt.show()

In [ ]:
# Проверим, сколько пропущенных данных и какие
total_missing = train_data.isna().sum().sum()
total_cell = np.product(train_data.shape)

print(f"missing values: {(total_missing / total_cell) * 100}%")

train_data.isna().mean()

In [ ]:
# Заполним пропуски
# train_data["build_tech"] = train_data["build_tech"].fillna(0)
# train_data.loc[train_data["build_tech"] > 1, "build_tech"] = 1

# train_data["metro_dist"] = train_data["metro_dist"].fillna(train_data["metro_dist"].mean())

# cond1 = train_data['floor'] > 8
# cond2 = train_data['floor'] <= 8
# train_data.loc[cond1,'g_lift'] = train_data.loc[cond1,'g_lift'].fillna(1.)
# train_data.loc[cond2,'g_lift'] = train_data.loc[cond2,'g_lift'].fillna(0.)

total_missing = train_data.isna().sum().sum()
total_cell = np.product(train_data.shape)

print(f"missing values: {(total_missing / total_cell) * 100}%")

# Удаляем выбросы
# train_data = train_data.drop(train_data[(train_data['price'] > 80000000) & (train_data['build_tech'] == 2)].index)
train_data = train_data[train_data['price'] < 70000000]

In [ ]:
# Аналогично заполним пропуски для test_data
# test_data["build_tech"] = test_data["build_tech"].fillna(0)

# test_data["metro_dist"] = test_data["metro_dist"].fillna(test_data["metro_dist"].mean())

# cond1 = test_data['floor'] > 8
# cond2 = test_data['floor'] <= 8
# test_data.loc[cond1,'g_lift'] = test_data.loc[cond1,'g_lift'].fillna(1.)
# test_data.loc[cond2,'g_lift'] = test_data.loc[cond2,'g_lift'].fillna(0.)

In [ ]:
def date_features(df):
    df["quarter"] = df.date.apply(lambda s: s.quarter)
    df["month"] = df.date.apply(lambda s: s.month)
    df["week"] = df.date.apply(lambda s: s.week)
    df["day"] = df.date.apply(lambda s: s.day)
    df["dayofweek"] = df.date.apply(lambda s: s.dayofweek)
    
    return df

In [ ]:
train_data.date = pd.to_datetime(train_data.date)
test_data.date = pd.to_datetime(test_data.date)

train_data = date_features(train_data)
test_data = date_features(test_data)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
ax.scatter(x = train_data['area'], y = train_data['price'])
plt.xlabel('area', fontsize=12)
plt.ylabel('price', fontsize=12)
plt.show()

In [ ]:
# Сохраняем колонку с 'Id'
# train_ID = train_data['id']
# test_ID = test_data['id']



Y = train_data.price
X = train_data.drop(["id", "date", "price"] , axis = 1)


test_X = test_data.drop(["id", "date"], axis = 1)


In [ ]:
tss = TimeSeriesSplit()

for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    
x_train, x_test, y_train, y_test = X_train, X_test, Y_train, Y_test

In [ ]:
X_test = test_X

reg1 = XGBRegressor(
    max_depth=12,
    n_estimators=9000,
    min_child_weight=6,
    colsample_bytree=1, 
    subsample=1, 
    eta=1.4,
    n_jobs=6,
    booster='gbtree',
    reg_alpha=1,
    reg_lambda=1,
    random_state=2,
    learning_rate=0.1
)

reg1.fit(
    x_train, 
    y_train, 
    eval_metric='rmse', 
    eval_set=[(x_train, y_train), (x_test, y_test)], 
    early_stopping_rounds = 10)

# reg2 = RandomForestRegressor(
#     n_estimators=9000, #
#     criterion='mse', # 
#     max_depth=12, #
#     min_samples_split=12, 
#     min_samples_leaf=6, 
#     max_features='auto', # 
#     bootstrap=True, #
#     oob_score=False, #
#     n_jobs=6, # 
#     random_state=2 #
# )


# Создаём ансамбль моделей
# ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2)])

# Обучаем ансамбль
# ereg = ereg.fit(
#     x_train,
#     y_train
# )

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(sample.shape)

In [ ]:
# Делаем предсказания
y_pred = reg1.predict(X_test)

sample['price'] = y_pred
sample.to_csv("res.csv", index=False)